## Librerias

In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import SimpleImputer
import seaborn as sns
from scipy.stats import chi2_contingency
import scipy.stats as ss
import os
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Funciones

In [2]:
def agrupar_categorias_pequenas(df, columna, umbral=100):
    # Contar la frecuencia de cada categoría
    conteo_categorias = df[columna].value_counts()
    
    # Identificar categorías con observaciones por debajo del umbral
    categorias_pequenas = conteo_categorias[conteo_categorias < umbral].index
    
    # Mantener el mismo tipo de las categorías originales
    tipo_original = df[columna].dtype.type
    
    # Reemplazar esas categorías con 'Otros', asegurando el mismo tipo de dato
    df[columna] = df[columna].apply(lambda x: tipo_original('Otros') if x in categorias_pequenas else x)
    
    return df

def eliminar_categorias_sin_conversion(df, columna, label_col='Label'):
    # Identificar las categorías donde Label = 1 no está presente
    categorias_a_eliminar = df.groupby(columna)[label_col].sum()[df.groupby(columna)[label_col].sum() == 0].index
    # Eliminar esas categorías
    df = df[~df[columna].isin(categorias_a_eliminar)]
    return df

def cramers_v(confusion_matrix):
    chi2 = ss.chi2_contingency(confusion_matrix, correction=False)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    if min(r, k) == 1:
        return np.nan  # Evita dividir por cero
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

def expand_auction_list(df, column):
    # Eliminar la columna original y reemplazarla con las nuevas columnas
    categories = set()
    
    # Recorrer todas las listas y agregar cada categoría única al conjunto
    for row in df[column]:
        if pd.notna(row):
            categories.update(eval(row))  # eval transforma la string en lista
    
    # Crear un DataFrame temporal para almacenar las nuevas columnas
    new_columns = pd.DataFrame()
    
    # Llenar el DataFrame temporal con las nuevas columnas
    for category in categories:
        new_columns[category] = df[column].apply(lambda x: 1 if pd.notna(x) and category in eval(x) else 0)
    
    # Concatenar el nuevo DataFrame con las columnas originales
    df = pd.concat([df, new_columns], axis=1)
    
    # Eliminar la columna original
    df = df.drop(columns=[column])
    
    return df

def data_cleaning(df, columns_to_drop):
    # Eliminar las columnas del dataset
    for col in columns_to_drop:
        if col in df.columns:
            df = df.drop(columns=[col])

    categorical_features = [col for col in df.select_dtypes(include=['object']).columns.tolist() if col != 'auction_list_0' and col != 'auction_list_1' and col != 'auction_list_2']

    for col in categorical_features:
        df = agrupar_categorias_pequenas(df, col, umbral=100) 

    for col in categorical_features:
        df = eliminar_categorias_sin_conversion(df, col)

    # Aplicar la función para la columna auction_list_0
    df = expand_auction_list(df, 'auction_list_0')
    df = expand_auction_list(df, 'action_list_1')
    df = expand_auction_list(df, 'action_list_2')

    return df

## Data

In [3]:
# train_data_21 = pd.read_csv("data/ctr_21.csv")
# train_data_20 = pd.read_csv("data/ctr_20.csv")
# train_data_19 = pd.read_csv("data/ctr_19.csv")
# train_data_18 = pd.read_csv("data/ctr_18.csv")
# train_data_17 = pd.read_csv("data/ctr_17.csv")
# train_data_16 = pd.read_csv("data/ctr_16.csv")
# train_data_15 = pd.read_csv("data/ctr_15.csv")
# train_data_combined = pd.concat([train_data_21, train_data_20, train_data_19, train_data_18, train_data_17, train_data_16, train_data_15], axis=0)
# train_data_combined = pd.concat([train_data_21, train_data_20], axis=0)
train_data_combined = pd.read_csv("data/ctr_21.csv")

In [4]:
random_state = 43992294

## Caracteristicas principales

In [ ]:
# Imprimir la cantidad de filas del dataset combinado
print(f"Cantidad de filas en el dataset combinado: {train_data_combined.shape[0]}")

# Ver porcentaje de clics vs no clics en la columna Label
label_counts = train_data_combined['Label'].value_counts(normalize=True) * 100
print("\nPorcentaje de clics (1) y no clics (0):")
print(label_counts)

# Cantidad de clics (1) y no clics (0)
label_counts_abs = train_data_combined['Label'].value_counts()
print("\nCantidad de clics (1) y no clics (0):")
print(label_counts_abs)

In [ ]:
# Resumen estadístico de las características numéricas
print(train_data_combined.describe())

# Resumen de las características categóricas
print(train_data_combined.describe(include='object'))

In [13]:
# Ver el porcentaje de valores faltantes por columna
missing_data = train_data_combined.isnull().mean() * 100
print(missing_data[missing_data > 0].sort_values(ascending=False))

creative_categorical_3     95.396400
creative_categorical_6     82.821600
creative_categorical_7     82.476016
creative_categorical_12    82.288322
auction_age                80.752629
creative_categorical_5     79.146887
creative_categorical_2     78.629825
gender                     77.193460
action_list_1              36.057013
auction_categorical_9      27.289868
action_list_2              27.067379
creative_categorical_4     20.853113
creative_width             18.358662
creative_height            18.358662
auction_categorical_6       7.863271
auction_boolean_1           2.912906
auction_boolean_2           2.613081
auction_boolean_0           1.492101
auction_list_0              1.481991
creative_categorical_9      0.874266
auction_categorical_2       0.833300
auction_categorical_0       0.461785
auction_categorical_4       0.393705
timezone_offset             0.320702
auction_categorical_3       0.319717
auction_categorical_11      0.157298
auction_categorical_7       0.002823
a

In [5]:
# Dividir los datos en numéricos y categóricos
numeric_features = train_data_combined.select_dtypes(include=['number']).columns.tolist()

categorical_features = train_data_combined.select_dtypes(include=['object']).columns.tolist()

## Columna por columna

In [6]:
# Iterar sobre cada columna y mostrar los valores únicos
for column in train_data_combined.columns:
    unique_values = train_data_combined[column].unique()
    print(f"Columna: {column}")
    print(f"Valores únicos ({len(unique_values)}): {unique_values}")
    print("-" * 50)  # Separador entre columnas

Columna: Label
Valores únicos (2): [0 1]
--------------------------------------------------
Columna: action_categorical_0
Valores únicos (9): ['e350c7c7' '9915ffee' 'c202ab08' '604d011f' 'c2e4f717' '11b7af3d'
 '6b5513a4' 'c186959e' '7f02cacc']
--------------------------------------------------
Columna: action_categorical_1
Valores únicos (14): ['e220fce9' 'd9eb38d8' '6d1ca31b' 'f71d2f9b' '3074db21' 'ac0f362d'
 'dc24b79b' '0d132da2' '49bef539' '62f19448' 'e709bbc0' '11fe6f26'
 'df5eb4dc' 'f4cd321d']
--------------------------------------------------
Columna: action_categorical_2
Valores únicos (96): ['5a39c3f0' 'c0f33c37' 'f44a466a' '2e6c63bf' '3786e6dc' '1ae7a73f'
 '0ce6d1d4' '2f2b9920' '7c0ac8b2' '8cc5b58e' '20ae8708' '090f19bf'
 'c54e86fa' 'b81de479' '8f1154ef' '40ef304e' 'f9c44bd9' 'b2fc4ccf'
 'bef86fd1' 'a6301ada' 'ec9345bf' 'd2f34a41' '880f2cbf' '8ec192a6'
 '84284433' '7a4ae307' '519a1d40' '478b337e' '35fedca8' '8a6fe935'
 '11b679f6' '83fa35e8' '9a115291' '75f68f1d' 'e6bf2018' '1a

In [7]:
# Iterar sobre cada columna y mostrar los valores únicos y la cantidad de veces que aparecen
for column in train_data_combined.columns:
    print(f"Columna: {column}")
    print(f"Valores únicos y su frecuencia:")
    print(train_data_combined[column].value_counts())
    print("-" * 50)  # Separador entre columnas

Columna: Label
Valores únicos y su frecuencia:
Label
0    1501392
1      21829
Name: count, dtype: int64
--------------------------------------------------
Columna: action_categorical_0
Valores únicos y su frecuencia:
action_categorical_0
604d011f    405936
9915ffee    355749
11b7af3d    257867
c2e4f717    211808
e350c7c7    129879
c202ab08    123665
6b5513a4     21268
7f02cacc     10689
c186959e      6360
Name: count, dtype: int64
--------------------------------------------------
Columna: action_categorical_1
Valores únicos y su frecuencia:
action_categorical_1
f71d2f9b    381488
d9eb38d8    235340
3074db21    202144
ac0f362d    156814
e220fce9    129879
6d1ca31b    123665
dc24b79b    120409
0d132da2    101053
49bef539     24448
62f19448     15391
f4cd321d     10689
e709bbc0      9664
11fe6f26      6360
df5eb4dc      5877
Name: count, dtype: int64
--------------------------------------------------
Columna: action_categorical_2
Valores únicos y su frecuencia:
action_categorical_2
3786

In [9]:
# Inicializar un conjunto vacío para almacenar los entity ids únicos
unique_entity_ids_2 = set()

# Recorrer cada fila de la columna 'action_list_2'
for lista in train_data_combined['action_list_2'].dropna():
    # Convertir el string de la lista a una lista real y agregar cada valor al conjunto
    unique_entity_ids_2.update(eval(lista))  # eval transforma la string en lista si es necesario

# Convertir el conjunto en una lista ordenada
unique_entity_ids_2 = sorted(unique_entity_ids_2)

# Mostrar los entity ids únicos
print(unique_entity_ids_2)



[-7265, -7264, -7263, -7196, -7195, -7190, -7143, -7127, -7126, -7112, -7111, -6946, -6938, -6929, -6905, -6904, -6876, -6875, -6871, -6867, -6866, -6865, -6850, -6849, -6848, -6823, -6800, -6780, -6775, -6774, -6770, -6621, -6620, -6618, -6617, -6615, -6548, -6547, -6544, -6543, -6454, -6309, -6226, -6224, -6223, -6220, -6219, -6218, -6217, -6125, -6119, -6118, -5902, -5613, -5605, -5604, -5603, -5579, -5578, -5577, -5576, -5560, -5559, -5471, -5470, -5469, -2560, 2560, 5469, 5470, 5471, 5559, 5560, 5576, 5577, 5578, 5579, 5603, 5604, 5605, 5613, 5902, 6118, 6119, 6125, 6217, 6218, 6219, 6220, 6223, 6224, 6226, 6309, 6451, 6454, 6543, 6544, 6547, 6548, 6615, 6616, 6618, 6620, 6621, 6770, 6774, 6775, 6780, 6800, 6823, 6848, 6849, 6850, 6865, 6866, 6867, 6871, 6875, 6876, 6904, 6905, 6929, 6938, 6946, 7111, 7112, 7126, 7127, 7143, 7190, 7195, 7196, 7263, 7264, 7265]


In [11]:
print(len(unique_entity_ids_2))

135


In [10]:
# Inicializar un conjunto vacío para almacenar los entity ids únicos
unique_entity_ids = set()

# Recorrer cada fila de la columna 'action_list_1'
for lista in train_data_combined['action_list_1'].dropna():
    # Convertir el string de la lista a una lista real y agregar cada valor al conjunto
    unique_entity_ids.update(eval(lista))  # eval transforma la string en lista si es necesario

# Convertir el conjunto en una lista ordenada
unique_entity_ids = sorted(unique_entity_ids)

# Mostrar los entity ids únicos
print(unique_entity_ids)

[-7199, -7195, -7194, -7190, -7033, -6903, -6902, -6874, -6871, -6824, -6823, -6780, -6779, -6618, -6617, -6616, -6615, -6614, -6613, -6454, -6451, -6119, -6118, -5736, -5579, -5578, -5577, -5559, -2606, 2606, 5559, 5577, 5578, 5579, 5736, 6118, 6119, 6451, 6454, 6614, 6615, 6616, 6617, 6618, 6779, 6780, 6823, 6824, 6871, 6874, 6902, 6903, 7033, 7190, 7194, 7195, 7199]


In [12]:
print(len(unique_entity_ids))

57


## Covarianzas

In [ ]:
# Top Covariances 
covs = train_data_combined[numeric_features].cov()
# Fill diagonal with 0s
for i in range(len(covs)):
    covs.iloc[i,i] = 0

covs.unstack().sort_values(ascending=False).drop_duplicates()

En el análisis de covarianzas, se observan relaciones fuertes entre **auction_time** y **creative_width**, así como entre **creative_height** y **creative_width**, lo que sugiere posible redundancia en estas variables. Por otro lado, las covarianzas más bajas, especialmente entre **Label** y variables como **creative_height** y **creative_width**, indican que estas características podrían no estar aportando mucho valor predictivo a la variable objetivo. Como consecuencia, sería recomendable dropear tanto **creative_height** como **creative_width** para simplificar el modelo y reducir la dimensionalidad sin perder información relevante. En resumen, estas observaciones sugieren la posible eliminación de **creative_height** y **creative_width**, y quizás una revisión más detallada de la relación entre **auction_time** y otras variables como **auction_bidfloor** y **auction_age** para identificar si aportan un valor significativo en el modelo.

Debido a que la variable objetivo **Label** esta desbalanceada, ssamos *Stratification* para dividir el dataset en train y test, de manera que la distribucion de la variable objetivo sea la misma en ambos conjuntos.

## Correlaciones

In [ ]:
# Calcular la correlación solo entre las variables numéricas
correlations = train_data_combined[numeric_features].corr()

# Mostrar las correlaciones de las variables numéricas con la variable objetivo 'Label'
print(correlations['Label'].sort_values(ascending=False))


En base a estas correlaciones, podrías considerar eliminar variables como creative_width, creative_height, auction_time, y auction_age, especialmente si no aportan valor en modelos más complejos. Sin embargo, antes de hacerlo, sería útil entrenar un modelo y verificar su rendimiento con y sin estas características para confirmar su utilidad. Algunas variables como auction_bidfloor y creative_height, aunque tienen correlaciones bajas, podrían ser útiles en un modelo no lineal como Random Forest o XGBoost, donde las relaciones más complejas entre las variables podrían ser capturadas.

In [ ]:
# Matriz de correlación de las características numéricas
plt.figure(figsize=(10, 8))
sns.heatmap(correlations, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de correlación')
plt.show()

In [ ]:
# Diccionario para guardar las correlaciones
categorical_correlations = {}

# Calcular el V de Cramer para cada variable categórica
for column in categorical_features:
    if column != 'Label':  # Excluir 'Label'
        confusion_matrix = pd.crosstab(train_data_combined[column], train_data_combined['Label'])
        categorical_correlations[column] = cramers_v(confusion_matrix)

# Convertir el diccionario en un DataFrame para facilitar la visualización
correlation_df = pd.DataFrame.from_dict(categorical_correlations, orient='index', columns=['CramersV'])

# Visualizar las correlaciones con un gráfico de barras
plt.figure(figsize=(10, 8))
correlation_df.sort_values(by='CramersV', ascending=False).plot(kind='bar', legend=False)
plt.title('Correlación de Atributos Categóricos con la Variable Objetivo (Label)')
plt.xlabel('Atributos Categóricos')
plt.ylabel('V de Cramer')
plt.tight_layout()
plt.show()

# Convertir el DataFrame de correlaciones en una lista ordenada
correlation_list = correlation_df.sort_values(by='CramersV', ascending=False).reset_index().values.tolist()

# Mostrar la lista
correlation_list

## Data cleaning

In [24]:
random_state = 43992294

In [71]:
# Lista de columnas adicionales a eliminar
columns_to_drop = [
    'creative_categorical_3', 'creative_categorical_6', 'creative_categorical_7', 
    'creative_categorical_12', 'auction_age', 'creative_categorical_5', 
    'creative_categorical_2', 'auction_boolean_1', 'auction_boolean_2', 
    'auction_boolean_0', 'action_categorical_6', 'action_categorical_7', 
    'auction_categorical_10', 'auction_categorical_2', 'creative_height', 
    'creative_width', 'auction_time', 'timezone_offset', 'auction_bidfloor'
]

In [75]:
# Iterar sobre cada columna y mostrar los valores únicos
for column in train_data_combined.columns:
    unique_values = train_data_combined[column].unique()
    print(f"Columna: {column}")
    print(f"Valores únicos ({len(unique_values)}): {unique_values}")
    print("-" * 50)  # Separador entre columnas

Columna: Label
Valores únicos (2): [0 1]
--------------------------------------------------
Columna: action_categorical_0
Valores únicos (9): ['e350c7c7' '9915ffee' 'c202ab08' '604d011f' 'c2e4f717' '11b7af3d'
 '6b5513a4' 'c186959e' '7f02cacc']
--------------------------------------------------
Columna: action_categorical_1
Valores únicos (14): ['e220fce9' 'd9eb38d8' '6d1ca31b' 'f71d2f9b' '3074db21' 'ac0f362d'
 'dc24b79b' '0d132da2' '49bef539' '62f19448' 'e709bbc0' '11fe6f26'
 'df5eb4dc' 'f4cd321d']
--------------------------------------------------
Columna: action_categorical_2
Valores únicos (85): ['5a39c3f0' 'c0f33c37' 'f44a466a' '2e6c63bf' '3786e6dc' '1ae7a73f'
 '0ce6d1d4' '2f2b9920' '7c0ac8b2' '8cc5b58e' '20ae8708' '090f19bf'
 'c54e86fa' 'b81de479' '8f1154ef' '40ef304e' 'f9c44bd9' 'b2fc4ccf'
 'bef86fd1' 'a6301ada' 'ec9345bf' 'd2f34a41' '880f2cbf' '8ec192a6'
 '84284433' '7a4ae307' '519a1d40' '478b337e' '35fedca8' '8a6fe935'
 '11b679f6' '83fa35e8' '9a115291' '75f68f1d' 'e6bf2018' '1a

In [ ]:
train_data_combined = data_cleaning(train_data_combined, columns_to_drop)

In [5]:
# Guardar el dataset modificado como CSV antes de aplicar One-Hot Encoding
train_data_combined.to_csv('train_data_cleaned.csv', index=False)

In [9]:
train_data_combined.columns.tolist()

In [16]:
# Iterar sobre cada columna y mostrar los valores únicos y la cantidad de veces que aparecen
for column in train_data_combined.columns:
    print(f"Columna: {column}")
    print(f"Valores únicos y su frecuencia:")
    print(train_data_combined[column].value_counts())
    print("-" * 50)  # Separador entre columnas


# Encodings

## Target Encoding

In [ ]:
# Dividir los datos en entrenamiento y validación con estratificación
X_train, X_val, y_train, y_val = train_test_split(
    train_data_combined.drop(columns='Label'),  # Características
    train_data_combined['Label'],               # Variable objetivo
    test_size=0.2,                              # 20% para validación
    stratify=train_data_combined['Label'],      # Estratificación basada en la variable objetivo
    random_state=random_state                   # Semilla para reproducibilidad
)

# Verificar la proporción de clases en el conjunto de entrenamiento y validación
print("Proporción en el conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True))

print("Proporción en el conjunto de validación:")
print(y_val.value_counts(normalize=True))


In [ ]:
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

# Crear el Target Encoder para las columnas categóricas
target_encoder = ce.TargetEncoder(cols=categorical_features)

# Ajustar y transformar los datos de entrenamiento
X_train_encoded = target_encoder.fit_transform(X_train, y_train)

# Transformar los datos de validación
X_val_encoded = target_encoder.transform(X_val)

# Crear un imputador que llene los valores NaN con la media de cada columna
imputer = SimpleImputer(strategy='mean')

# Aplicar el imputador a los datos de entrenamiento
X_train_encoded = imputer.fit_transform(X_train_encoded)

# También aplica el imputador al conjunto de validación
X_val_encoded = imputer.transform(X_val_encoded)

# Convertir los resultados a DataFrame y asignar las columnas originales
X_train_encoded = pd.DataFrame(X_train_encoded, columns=X_train.columns)
X_val_encoded = pd.DataFrame(X_val_encoded, columns=X_val.columns)

# Ver los primeros datos
print(X_train_encoded.head())
print(X_val_encoded.head())

In [ ]:
# Cargar los datos de testeo
test_data = pd.read_csv("data/ctr_test.csv")

# Eliminar las columnas no deseadas
test_data = test_data.drop(columns=columns_to_drop)

test_data_encoded = target_encoder.transform(test_data.drop(columns=["id"]))
test_data_encoded = imputer.transform(test_data_encoded)  # Imputar los valores faltantes en el set de testeo

# Convertir los resultados a DataFrame y asignar las columnas originales
test_data_encoded = pd.DataFrame(test_data_encoded, columns=test_data.drop(columns=["id"]).columns)

# comparar tamaño de columnas
print(X_train_encoded.columns)
print(test_data_encoded.columns)

## One Hot Encoding

In [5]:
# Lista de columnas que no deben someterse a One-Hot Encoding
excluded_columns = ['IAB-5', 'IAB7-15', 'IAB16', 'IAB5-11', 'IAB1-2', 'IAB6-5', 'IAB5-1', 'IAB2', 'IAB7-25', 
                    'IAB9-28', 'IAB5-7', 'IAB10-3', 'IAB7-17', 'IAB12-3', 'IAB14-6', 'IAB1-7', 'IAB7-27', 
                    'IAB10-4', 'IAB7-3', 'IAB7-45', 'IAB9-26', 'IAB7-8', 'IAB9-8', 'IAB7-20', 'IAB19-6', 
                    'IAB11', 'photography', 'IAB9-13', 'IAB9-16', 'IAB4-5', 'IAB17', 'IAB19-1', 'IAB19-5', 
                    'IAB9-15', 'finance', 'IAB5-9', 'IAB9-14', 'weather', 'IAB3', 'IAB9-10', 'IAB16-2', 
                    'IAB9-21', 'IAB7-34', 'IAB19-3', 'IAB9-1', 'IAB9-19', 'IAB7-44', 'IAB19-23', 'IAB14-1', 
                    'IAB19-32', 'IAB7-43', 'IAB10-5', 'IAB10-7', 'IAB19-19', 'IAB1-4', 'IAB7-29', 'IAB9-27', 
                    'IAB4', 'IAB7-6', 'IAB7', 'IAB1-1', 'IAB12-2', 'IAB19-26', 'IAB10', 'IAB2-14', 'productivity', 
                    'IAB7-12', 'IAB7-42', 'IAB2-19', 'IAB19-28', 'IAB7-22', 'lifestyle', 'IAB17-44', 'IAB10-1', 
                    'IAB3-3', 'IAB6-9', 'IAB22', 'IAB19-16', 'IAB7-30', 'IAB7-9', 'IAB19-14', 'IAB5-6', 'IAB9-23', 
                    'navigation', 'IAB9-7', 'music', 'IAB16-4', 'IAB19-27', 'IAB19-22', 'IAB1-3', 'IAB7-38', 
                    'education', 'IAB19-24', 'IAB19-2', 'IAB7-39', 'travel', 'IAB17-26', 'IAB7-19', 'IAB5', 
                    'IAB15-6', 'IAB8-6', 'IAB9-3', 'IAB1', 'IAB7-13', 'IAB7-24', 'IAB11-2', 'IAB18', 'IAB18-1', 
                    'IAB19-25', 'IAB19-29', 'healthcare_and_fitness', 'IAB20-1', 'IAB9-29', 'IAB3-5', 'IAB4-4', 
                    'IAB7-1', 'IAB9-17', 'IAB7-21', 'IAB7-11', 'IAB24', 'IAB7-7', 'IAB7-35', 'IAB19-8', 'IAB7-10', 
                    'IAB9-9', 'IAB19-11', 'IAB19-12', 'IAB9-11', 'IAB7-2', 'IAB2-12', 'IAB19-21', 'IAB17-27', 
                    'IAB7-37', 'IAB19-18', 'IAB7-31', 'IAB19-4', 'IAB7-26', 'IAB19-31', 'IAB9-5', 'IAB15', 'IAB7-18', 
                    'IAB6-3', 'IAB7-32', 'IAB7-16', 'IAB19-33', 'reference', 'IAB9-22', 'IAB7-40', 'utilities', 
                    'IAB5-8', 'IAB20', 'IAB13', 'IAB19-13', 'IAB11-4', 'IAB19-35', 'IAB14-3', 'IAB7-23', 'IAB19-34', 
                    'IAB19-10', 'books', 'games', 'IAB12-1', 'IAB19-20', 'IAB7-28', 'IAB9-4', 'IAB9-12', 'IAB2-17', 
                    'IAB9-20', 'IAB9-6', 'IAB18-3', 'IAB7-14', 'IAB13-7', 'IAB17-13', 'IAB15-10', 'IAB19-30', 
                    'IAB19-17', 'IAB6', 'IAB9-24', 'IAB10-2', 'IAB21', 'IAB17-39', 'IAB6-6', 'IAB19-36', 'IAB19', 
                    'IAB8', 'IAB7-5', 'IAB1-6', 'IAB17-1', 'IAB12', 'IAB17-12', 'IAB9-25', 'news', 'IAB7-33', 
                    'IAB19-7', 'IAB9-30', 'sports', 'IAB19-15', 'IAB1-5', 'IAB6-4', 'entertainment', 'business', 
                    'IAB14', 'IAB7-41', 'IAB9-2', 'IAB9-31', 'IAB9-18', 'IAB7-36', 'medical', 'IAB2-4', 'IAB14-7', 
                    'social_networking', 'IAB7-4', 'IAB2-8', 'IAB9', 'IAB20-3', 'IAB16-6']

In [6]:
train_data_combined = pd.read_csv('train_data_cleaned.csv')

In [7]:
# Dividir los datos en entrenamiento y validación con estratificación
X_train, X_val, y_train, y_val = train_test_split(
    train_data_combined.drop(columns='Label'),  # Características
    train_data_combined['Label'],               # Variable objetivo
    test_size=0.2,                              # 20% para validación
    stratify=train_data_combined['Label'],      # Estratificación basada en la variable objetivo
    random_state=random_state                   # Semilla para reproducibilidad
)

# Asegurarse de que X_train y X_val sean dataframes de pandas
X_train = pd.DataFrame(X_train)
X_val = pd.DataFrame(X_val)

# Asegurarse de que y_train y y_val sean series de pandas
y_train = pd.Series(y_train)
y_val = pd.Series(y_val)

In [8]:
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
categorical_features_to_encode = [col for col in categorical_features if col not in excluded_columns]
numeric_features = X_train.select_dtypes(include=['number']).columns.tolist()

# Crear el transformador para One-Hot Encoding
one_hot_encoder = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features_to_encode)  # Aplicar OneHotEncoder a categóricas
    ],
    remainder='passthrough'  # Las columnas no categóricas se mantienen como están
)

# Crear un pipeline para imputar valores faltantes y aplicar One-Hot Encoding
pipeline = Pipeline(steps=[
    ('imputer', ColumnTransformer(
        transformers=[
            ('num', SimpleImputer(strategy='mean'), numeric_features),  # Imputar valores numéricos faltantes
            ('cat', SimpleImputer(strategy='most_frequent'), categorical_features_to_encode)  # Imputar valores categóricos faltantes
        ],
        remainder='passthrough'
    )),
    ('onehot', one_hot_encoder)  # Aplicar One-Hot Encoding solo a las columnas seleccionadas
])

In [ ]:
# Ajustar el pipeline y transformar los datos de entrenamiento y validación
X_train_encoded = pipeline.fit_transform(X_train)
X_val_encoded = pipeline.transform(X_val)

# Obtener los nombres de las nuevas columnas generadas por el One-Hot Encoding
# Obtener el OneHotEncoder dentro del pipeline para obtener las columnas
onehot_encoder = pipeline.named_steps['onehot'].named_transformers_['cat']

# Obtener los nombres de las nuevas columnas
encoded_columns = onehot_encoder.get_feature_names_out(categorical_features_to_encode)

# Combinar las columnas numéricas con las nuevas columnas codificadas
new_columns = numeric_features + list(encoded_columns)

# Convertir los resultados en DataFrames con los nuevos nombres de las columnas
X_train_encoded = pd.DataFrame(X_train_encoded, columns=new_columns)
X_val_encoded = pd.DataFrame(X_val_encoded, columns=new_columns)

In [ ]:
# Definir el espacio de búsqueda de hiperparámetros
space = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [None, 10, 20, 30, 40, 50]),
    'max_samples': hp.uniform('max_samples', 0.5, 1.0),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4]),
    'bootstrap': True
}

# Función objetivo para Hyperopt
def objective(params):
    # Asegurarse de que 'max_samples' solo se usa si 'bootstrap' es True
    if not params['bootstrap']:
        params['max_samples'] = None  # No se puede utilizar max_samples si bootstrap es False
    
    # Definir el modelo con los hiperparámetros actuales
    model = RandomForestClassifier(**params, random_state=random_state)
    
    # Entrenar el modelo directamente ya que el Target Encoder e Imputador ya se aplicaron
    model.fit(X_train_encoded, y_train)

    # Hacer predicciones en el conjunto de validación
    y_pred_proba = model.predict_proba(X_val_encoded)[:, 1]

    # Calcular el AUC en el conjunto de validación
    auc = roc_auc_score(y_val, y_pred_proba)
    
    # Imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params}, AUC: {auc:.4f}")

    # Retornar el valor negativo del AUC ya que Hyperopt minimiza por defecto
    return {'loss': 1-auc, 'status': STATUS_OK}

# Ejecutar la optimización
trials = Trials()  # Para guardar información sobre cada iteración
best = fmin(fn=objective, 
            space=space, 
            algo=tpe.suggest, 
            max_evals=50,  # Número de evaluaciones
            trials=trials)

print("Mejores hiperparámetros encontrados:")
print(best)

In [15]:
# Paso 5: Ajustar el pipeline y transformar los datos de entrenamiento y validación
X_train_encoded = pipeline.fit_transform(X_train)
X_val_encoded = pipeline.transform(X_val)

# Paso 6: Guardar los datos procesados
X_train_encoded = pd.DataFrame(X_train_encoded)

X_val_encoded = pd.DataFrame(X_val_encoded)

In [ ]:
# Paso 1: Cargar los datos de testeo
test_data = pd.read_csv("data/ctr_test.csv")

# Paso 2: Eliminar las columnas no deseadas
test_data = test_data.drop(columns=columns_to_drop)

# Paso 3: Modificar la columna auction_list_0 (expansión de la lista en múltiples columnas)
test_data = expand_auction_list(test_data, 'auction_list_0')

# Paso 4: Asegurarse de que las columnas en el conjunto de testeo coincidan con las del conjunto de entrenamiento antes de One-Hot Encoding
# Si en el conjunto de entrenamiento algunas categorías están presentes pero no en el conjunto de testeo, puedes agregarlas como columnas con valor 0
for col in X_train_encoded.columns:
    if col not in test_data.columns:
        test_data[col] = 0

# Paso 5: Identificar las columnas categóricas a codificar con One-Hot Encoding
categorical_features_test = [col for col in test_data.select_dtypes(include=['object']).columns if col not in excluded_columns]

# Paso 6: Aplicar One-Hot Encoding al conjunto de testeo
test_data_encoded = one_hot_encoder.transform(test_data)

# Paso 7: Imputar valores faltantes en el conjunto de testeo
test_data_encoded = imputer.transform(test_data_encoded)

# Paso 8: Convertir los resultados a DataFrame y asignar las columnas originales
test_data_encoded = pd.DataFrame(test_data_encoded, columns=X_train_encoded.columns)

# Comparar el tamaño de las columnas
print(f"Columnas en el conjunto de entrenamiento: {X_train_encoded.shape[1]}")
print(f"Columnas en el conjunto de testeo: {test_data_encoded.shape[1]}")

# Busqueda de hiperparametros

## Random Forest

### Random search hiperparametros

In [ ]:
# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'max_samples': np.linspace(0.5, 1.0, num=10),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Definir el modelo
model = RandomForestClassifier(random_state=random_state)

# Definir el RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=50,  # Número de combinaciones de hiperparámetros a probar
    scoring='roc_auc',
    cv=3,  # Número de folds en la validación cruzada
    verbose=2,
    random_state=random_state,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Ajustar el modelo a los datos de entrenamiento
random_search.fit(X_train_encoded, y_train)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(random_search.best_params_)

# Predecir en el conjunto de validación con el mejor modelo encontrado
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_val_encoded)[:, 1]

# Calcular el AUC en el conjunto de validación
auc = roc_auc_score(y_val, y_pred_proba)
print(f"AUC en el conjunto de validación: {auc:.4f}")

### Hyperopt

In [ ]:
# Definir el espacio de búsqueda de hiperparámetros
space = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [None, 10, 20, 30, 40, 50]),
    'max_samples': hp.uniform('max_samples', 0.5, 1.0),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4]),
    'bootstrap': True
}

# Función objetivo para Hyperopt
def objective(params):
    # Asegurarse de que 'max_samples' solo se usa si 'bootstrap' es True
    if not params['bootstrap']:
        params['max_samples'] = None  # No se puede utilizar max_samples si bootstrap es False
    
    # Definir el modelo con los hiperparámetros actuales
    model = RandomForestClassifier(**params, random_state=random_state)
    
    # Entrenar el modelo directamente ya que el Target Encoder e Imputador ya se aplicaron
    model.fit(X_train_encoded, y_train)

    # Hacer predicciones en el conjunto de validación
    y_pred_proba = model.predict_proba(X_val_encoded)[:, 1]

    # Calcular el AUC en el conjunto de validación
    auc = roc_auc_score(y_val, y_pred_proba)
    
    # Imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params}, AUC: {auc:.4f}")

    # Retornar el valor negativo del AUC ya que Hyperopt minimiza por defecto
    return {'loss': 1-auc, 'status': STATUS_OK}

# Ejecutar la optimización
trials = Trials()  # Para guardar información sobre cada iteración
best = fmin(fn=objective, 
            space=space, 
            algo=tpe.suggest, 
            max_evals=50,  # Número de evaluaciones
            trials=trials)

print("Mejores hiperparámetros encontrados:")
print(best)

## XGBoost

### Hyperopt

In [ ]:
# Cargar los datos (asegúrate de que sean DataFrames de pandas)
# X_train, X_val, y_train, y_val se deben definir previamente

# Espacio de búsqueda para los hiperparámetros
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'min_child_weight': hp.uniform('min_child_weight', 1, 10),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0.5, 2)
}

# Función objetivo para Hyperopt
def objective(params):
    # Entrenar el modelo con los hiperparámetros actuales
    model = xgb.XGBClassifier(
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        gamma=params['gamma'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        scale_pos_weight=params['scale_pos_weight'],
        use_label_encoder=False,
        eval_metric='auc',
        random_state=random_state
    )
    
    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train_encoded, y_train)
    
    # Predecir las probabilidades para calcular el AUC en el conjunto de validación
    y_pred_proba = model.predict_proba(X_val_encoded)[:, 1]
    auc = roc_auc_score(y_val, y_pred_proba)
    
    # Imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params}, AUC: {auc:.4f}")
    
    return {'loss': -auc, 'status': STATUS_OK}

# Ejecutar la optimización
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,  # Número de evaluaciones
            trials=trials)

print("Mejores hiperparámetros encontrados:")
print(best)

# Reentrenar el modelo con los mejores hiperparámetros
best_model = xgb.XGBClassifier(
    max_depth=int(best['max_depth']),
    learning_rate=best['learning_rate'],
    n_estimators=int(best['n_estimators']),
    gamma=best['gamma'],
    min_child_weight=best['min_child_weight'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    scale_pos_weight=best['scale_pos_weight'],
    use_label_encoder=False,
    eval_metric='auc',
    random_state=42
)

# Entrenar el modelo con los mejores hiperparámetros en los datos completos
best_model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de validación
y_pred_proba_val = best_model.predict_proba(X_val)[:, 1]
auc_val = roc_auc_score(y_val, y_pred_proba_val)
print(f"AUC en el conjunto de validación: {auc_val:.4f}")


## Pueba de Hiperparametros

### A mano

In [ ]:
# Reconstruir el modelo con los mejores hiperparámetros
best_model_rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=20,
    max_samples=None,
    min_samples_split=5,
    min_samples_leaf=4,
    bootstrap=False,
    random_state=random_state
)

# Crear el pipeline con el mejor modelo
rf_pipeline = make_pipeline(SimpleImputer(), best_model_rf)

# Entrenar el modelo con los datos de entrenamiento
rf_pipeline.fit(X_train_encoded, y_train)

In [ ]:
# Predecir en el conjunto de testeo
y_preds_rf_test = rf_pipeline.predict_proba(test_data_encoded)[:, 1]

# Crear el archivo de envío
submission_df_rf = pd.DataFrame({"id": test_data["id"], "Label": y_preds_rf_test})
submission_df_rf["id"] = submission_df_rf["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "random_forest"
file_name_rf = (
    f"random_forest_predictions_n_estimators_{500}_"
    f"max_depth_{20}_"
    f"min_samples_split_{5}_"
    f"min_samples_leaf_{4}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_rf.to_csv(os.path.join("submits", file_name_rf), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_rf}")

### Automatizada

In [ ]:
# Cargar los datos de testeo
test_data = pd.read_csv("data/ctr_test.csv")

# Convertir los hiperparámetros a enteros
best['n_estimators'] = int(best['n_estimators'])
best['min_samples_split'] = int(best['min_samples_split'])
best['min_samples_leaf'] = int(best['min_samples_leaf'])

# Reconstruir el modelo con los mejores hiperparámetros
best_model_rf = RandomForestClassifier(
    n_estimators=best['n_estimators'],
    max_depth=[None, 10, 20, 30, 40, 50][best['max_depth']],  # Hyperopt devuelve índices para opciones de lista
    max_samples=best['max_samples'],  # Para max_samples no es un índice, es un valor continuo
    min_samples_split=best['min_samples_split'],
    min_samples_leaf=best['min_samples_leaf'],
    bootstrap=[True, False][best['bootstrap']],  # Igual que max_depth
    random_state=random_state
)

# Crear el pipeline con el mejor modelo
rf_pipeline = make_pipeline(SimpleImputer(), best_model_rf)

# Entrenar el modelo con los datos de entrenamiento
rf_pipeline.fit(X_train_encoded, y_train)

# Predecir en el conjunto de testeo
test_data_encoded = target_encoder.transform(test_data.select_dtypes(include='number').drop(columns=["id"]))
test_data_encoded = imputer.transform(test_data_encoded)  # Imputar los valores faltantes en el set de testeo
y_preds_rf_test = rf_pipeline.predict_proba(test_data_encoded)[:, 1]

# Crear el archivo de envío
submission_df_rf = pd.DataFrame({"id": test_data["id"], "Label": y_preds_rf_test})
submission_df_rf["id"] = submission_df_rf["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "random_forest"
file_name_rf = (
    f"random_forest_predictions_n_estimators_{best['n_estimators']}_"
    f"max_depth_{best['max_depth']}_"
    f"min_samples_split_{best['min_samples_split']}_"
    f"min_samples_leaf_{best['min_samples_leaf']}_"
    f"AUC_{auc_rf_val:.4f}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_rf.to_csv(os.path.join("submits", file_name_rf), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_rf}")
